# 10.1 GroupBy Mechanics

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                  'key2': ['one', 'two', 'one', 'two', 'one'],
                  'data1': np.random.randn(5),
                  'data2': np.random.randn(5)})
df

,data1,data2,key1,key2
0,-2.062943,-0.739277,a,one
1,1.238172,-1.233857,a,two
2,-0.314761,2.076757,b,one
3,0.398332,-1.072538,b,two
4,-0.904080,0.104973,a,one


In [4]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [5]:
grouped.mean()

key1
a   -0.576284
b    0.041786
Name: data1, dtype: float64

In [9]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -1.483512
      two     1.238172
b     one    -0.314761
      two     0.398332
Name: data1, dtype: float64

In [10]:
means.unstack()

key2,one,two
key1,,
a,-1.483512,1.238172
b,-0.314761,0.398332


In [11]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005    1.238172
            2006   -0.314761
Ohio        2005   -0.832305
            2006   -0.904080
Name: data1, dtype: float64

In [12]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.576284,-0.622720
b,0.041786,0.502109


In [13]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

## Iterating Over Groups

In [14]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
      data1     data2 key1 key2
0 -2.062943 -0.739277    a  one
1  1.238172 -1.233857    a  two
4 -0.904080  0.104973    a  one
b
      data1     data2 key1 key2
2 -0.314761  2.076757    b  one
3  0.398332 -1.072538    b  two


In [15]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
      data1     data2 key1 key2
0 -2.062943 -0.739277    a  one
4 -0.904080  0.104973    a  one
('a', 'two')
      data1     data2 key1 key2
1  1.238172 -1.233857    a  two
('b', 'one')
      data1     data2 key1 key2
2 -0.314761  2.076757    b  one
('b', 'two')
      data1     data2 key1 key2
3  0.398332 -1.072538    b  two


In [16]:
pieces = dict(list(df.groupby('key1')))
pieces

{'a':       data1     data2 key1 key2
 0 -2.062943 -0.739277    a  one
 1  1.238172 -1.233857    a  two
 4 -0.904080  0.104973    a  one, 'b':       data1     data2 key1 key2
 2 -0.314761  2.076757    b  one
 3  0.398332 -1.072538    b  two}

In [17]:
pieces['b']

,data1,data2,key1,key2
2,-0.314761,2.076757,b,one
3,0.398332,-1.072538,b,two


In [18]:
df.dtypes

data1    float64
data2    float64
key1      object
key2      object
dtype: object

In [20]:
grouped = df.groupby(df.dtypes, axis=1)

In [24]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -2.062943 -0.739277
1  1.238172 -1.233857
2 -0.314761  2.076757
3  0.398332 -1.072538
4 -0.904080  0.104973
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


## Selecting a Column or Subset of Columns

In [26]:
df.groupby('key1')['data1'].mean()

key1
a   -0.576284
b    0.041786
Name: data1, dtype: float64

In [27]:
df.groupby('key1')[['data1']].mean()

,data1
key1,
a,-0.576284
b,0.041786


In [29]:
df['data1'].groupby(df['key1']).mean()

key1
a   -0.576284
b    0.041786
Name: data1, dtype: float64

In [30]:
df[['data1']].groupby(df['key1']).mean()

,data1
key1,
a,-0.576284
b,0.041786


## Grouping with Dicts and Series

In [31]:
people = pd.DataFrame(np.random.randn(5, 5),
                     columns=['a', 'b', 'c', 'd', 'e'],
                     index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan
people

,a,b,c,d,e
Joe,0.297501,-0.104347,-1.410354,0.273931,1.970058
Steve,2.957686,0.078042,0.227669,-1.260092,0.239937
Wes,0.187813,NaN,NaN,-0.858558,-0.063085
Jim,-0.252824,-0.149366,-0.846139,0.758721,-0.133504
Travis,-1.543056,-1.606644,0.594331,0.220904,-0.155826


In [32]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 
          'd': 'blue', 'e': 'red', 'f': 'orange'}
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,-1.136423,2.163213
Steve,-1.032423,3.275666
Wes,-0.858558,0.124728
Jim,-0.087418,-0.535694
Travis,0.815235,-3.305526


In [33]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [34]:
people.groupby(map_series, axis=1).sum()

,blue,red
Joe,-1.136423,2.163213
Steve,-1.032423,3.275666
Wes,-0.858558,0.124728
Jim,-0.087418,-0.535694
Travis,0.815235,-3.305526


In [35]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


## Grouping with Functions

In [36]:
people.groupby(len).sum()

,a,b,c,d,e
3,0.232490,-0.253713,-2.256492,0.174094,1.773469
5,2.957686,0.078042,0.227669,-1.260092,0.239937
6,-1.543056,-1.606644,0.594331,0.220904,-0.155826


In [37]:
people.groupby(len).count()

,a,b,c,d,e
3,3,2,2,3,3
5,1,1,1,1,1
6,1,1,1,1,1


In [38]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one  0.187813 -0.104347 -1.410354 -0.858558 -0.063085
  two -0.252824 -0.149366 -0.846139  0.758721 -0.133504
5 one  2.957686  0.078042  0.227669 -1.260092  0.239937
6 two -1.543056 -1.606644  0.594331  0.220904 -0.155826

## Grouping by Index Levels

In [44]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                   [1, 3, 5, 1, 3]],
                                   names=['city', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

city         US                            JP          
tenor         1         3         5         1         3
0      2.011105  0.225222 -0.056363  0.177508 -0.367334
1     -0.893285 -0.475074 -0.011017  0.155759 -2.593004
2      0.056838  1.152407 -0.700847 -0.291982  0.167883
3     -1.381393 -1.284217 -0.861549 -0.269369 -0.716634

In [46]:
hier_df.groupby(level='city', axis=1).count()

city,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


# 10.2 Data Aggregation

In [47]:
df

,data1,data2,key1,key2
0,-2.062943,-0.739277,a,one
1,1.238172,-1.233857,a,two
2,-0.314761,2.076757,b,one
3,0.398332,-1.072538,b,two
4,-0.904080,0.104973,a,one


In [48]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    0.809721
b    0.327023
Name: data1, dtype: float64

In [49]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [50]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,3.301115,1.338830
b,0.713093,3.149295


In [51]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.576284  1.674792 -2.062943 -1.483512 -0.904080  0.167046   
b      2.0  0.041786  0.504233 -0.314761 -0.136487  0.041786  0.220059   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.238172   3.0 -0.622720  0.676983 -1.233857 -0.986567 -0.739277   
b     0.398332   2.0  0.502109  2.226888 -1.072538 -0.285214  0.502109   

                          
           75%       max  
key1                      
a    -0.317152  0.104973  
b     1.289433  2.076757

## Column-Wise and Multiple Function Application

In [52]:
tips = pd.read_csv('examples/tips.csv')
tips.shape

(244, 6)

In [53]:
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [54]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [58]:
grouped = tips.groupby(['day', 'smoker'])
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [59]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [60]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [61]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [63]:
grouped.agg({'tip_pct': ['min', np.max, 'mean', 'std', np.var], 'size': 'sum'})

tip_pct                                         size
                  min      amax      mean       std       var  sum
day  smoker                                                       
Fri  No      0.120385  0.187735  0.151650  0.028123  0.000791    9
     Yes     0.103555  0.263480  0.174783  0.051293  0.002631   31
Sat  No      0.056797  0.291990  0.158048  0.039767  0.001581  115
     Yes     0.035638  0.325733  0.147906  0.061375  0.003767  104
Sun  No      0.059447  0.252672  0.160113  0.042347  0.001793  167
     Yes     0.065660  0.710345  0.187250  0.154134  0.023757   49
Thur No      0.072961  0.266312  0.160298  0.038774  0.001503  112
     Yes     0.090014  0.241255  0.163863  0.039389  0.001551   40

## Returning Aggregated Data Without Row Indexes

In [64]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [65]:
tips.groupby(['day', 'smoker']).mean()

total_bill       tip      size   tip_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863

# 10.3 Apply: General split-apply-combine

In [67]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]
top(tips)

,total_bill,tip,smoker,day,time,size,tip_pct
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [69]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [70]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [71]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [72]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

## Suppressing the Group Keys

In [74]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


## Quantile and Bucket Analysis

In [75]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4)
quartiles.head(10)

0    (-2.495, -0.619]
1     (-0.619, 1.256]
2     (-0.619, 1.256]
3     (-0.619, 1.256]
4    (-4.378, -2.495]
5    (-2.495, -0.619]
6     (-0.619, 1.256]
7     (-0.619, 1.256]
8     (-0.619, 1.256]
9      (1.256, 3.132]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-4.378, -2.495] < (-2.495, -0.619] < (-0.619, 1.256] < (1.256, 3.132]]

In [76]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
           'count': group.count(), 'mean': group.mean()}
grouped = frame.data2.groupby(quartiles)

In [77]:
grouped.apply(get_stats)

data1                  
(-4.378, -2.495]  count      6.000000
                  max        0.352628
                  mean      -0.513729
                  min       -1.273193
(-2.495, -0.619]  count    257.000000
                  max        2.183215
                  mean      -0.028542
                  min       -3.299105
(-0.619, 1.256]   count    627.000000
                  max        3.145292
                  mean      -0.061412
                  min       -2.923244
(1.256, 3.132]    count    110.000000
                  max        2.101830
                  mean       0.016974
                  min       -2.132375
Name: data2, dtype: float64

In [78]:
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-4.378, -2.495]",6.0,0.352628,-0.513729,-1.273193
"(-2.495, -0.619]",257.0,2.183215,-0.028542,-3.299105
"(-0.619, 1.256]",627.0,3.145292,-0.061412,-2.923244
"(1.256, 3.132]",110.0,2.101830,0.016974,-2.132375


In [79]:
grouping = pd.qcut(frame.data1, 10, labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,100.0,2.183215,-0.071481,-2.119732
1,100.0,2.167797,-0.041748,-3.299105
2,100.0,3.145292,0.073933,-2.417420
3,100.0,2.127597,-0.081459,-1.989854
4,100.0,2.959534,-0.097951,-2.865329
5,100.0,2.095932,-0.104785,-2.923244
6,100.0,2.724538,-0.017752,-2.341814
7,100.0,2.171264,-0.126453,-2.577715
8,100.0,2.641469,-0.016149,-1.983260


## Example: Filling Missing Values with Group-Speci c Values

In [80]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s.fillna(s.mean())

0    0.234361
1    0.499476
2    0.234361
3   -1.168051
4    0.234361
5    1.371657
dtype: float64

## Example: Random Sampling and Permutation

In [81]:
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1,11)) + [10]*3)*4
base_names = ['A'] + list(range(2, 11)) + ['J', 'Q', 'K']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)
deck

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
QH     10
KH     10
AS      1
2S      2
3S      3
4S      4
5S      5
6S      6
7S      7
8S      8
9S      9
10S    10
JS     10
QS     10
KS     10
AC      1
2C      2
3C      3
4C      4
5C      5
6C      6
7C      7
8C      8
9C      9
10C    10
JC     10
QC     10
KC     10
AD      1
2D      2
3D      3
4D      4
5D      5
6D      6
7D      7
8D      8
9D      9
10D    10
JD     10
QD     10
KD     10
dtype: int64

In [83]:
def draw(deck, n=5):
    return deck.sample(n)
draw(deck)

5C     5
KD    10
2C     2
9D     9
3D     3
dtype: int64

In [84]:
get_suit = lambda c: c[-1]
deck.groupby(get_suit).apply(draw, n=2)

C  4C     4
   JC    10
D  7D     7
   3D     3
H  8H     8
   9H     9
S  9S     9
   6S     6
dtype: int64

## Example: Group Weighted Average and Correlation

In [85]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})
df

,category,data,weights
0,a,-0.667362,0.982896
1,a,-0.688069,0.955709
2,a,1.426246,0.356548
3,a,-0.621686,0.287502
4,b,-0.210103,0.359062
5,b,-2.942433,0.540934
6,b,0.699770,0.095717
7,b,0.175218,0.342947


In [86]:
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

category
a   -0.380907
b   -1.150427
dtype: float64

In [89]:
close_px = pd.read_csv('examples/stock_px_2.csv', parse_dates=True,
                       index_col=0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
AAPL    2214 non-null float64
MSFT    2214 non-null float64
XOM     2214 non-null float64
SPX     2214 non-null float64
dtypes: float64(4)
memory usage: 86.5 KB


In [90]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [91]:
spx_corr = lambda x: x.corrwith(x['SPX'])
rets = close_px.pct_change().dropna()

In [92]:
get_year = lambda x: x.year
by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [93]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

## Example: Group-Wise Linear Regression

In [87]:
import statsmodels.api as sm

/Users/khwu/.virtualenvs/analytic/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [88]:
def regress(data, yvar, xvar):
    Y = data[yvar]
    X = data[xvar]
    X['intercept'] = 1
    result = sm.OLS(Y, X).fit()
    return result.params

In [94]:
by_year.apply(regress,'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


# 10.4 Pivot Tables and Cross-Tabulation

In [96]:
tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [97]:
tips.shape

(244, 7)

In [98]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [99]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [101]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [103]:
tips.pivot_table(['tip_pct'], index=['time', 'smoker'],
                columns='day', aggfunc=len, margins=True)

tip_pct                         
day               Fri   Sat   Sun  Thur    All
time   smoker                                 
Dinner No         3.0  45.0  57.0   1.0  106.0
       Yes        9.0  42.0  19.0   NaN   70.0
Lunch  No         1.0   NaN   NaN  44.0   45.0
       Yes        6.0   NaN   NaN  17.0   23.0
All              19.0  87.0  76.0  62.0  244.0

In [106]:
tips.pivot_table(['tip_pct'], index=['time', 'size','smoker'],
                columns='day', aggfunc=np.average, margins=True, fill_value=0)

tip_pct                                        
day                      Fri       Sat       Sun      Thur       All
time   size smoker                                                  
Dinner 1    No      0.000000  0.137931  0.000000  0.000000  0.137931
            Yes     0.000000  0.325733  0.000000  0.000000  0.325733
       2    No      0.139622  0.162705  0.168859  0.159744  0.164383
            Yes     0.171297  0.148668  0.207893  0.000000  0.167246
       3    No      0.000000  0.154661  0.152663  0.000000  0.153705
            Yes     0.000000  0.144995  0.152660  0.000000  0.148061
       4    No      0.000000  0.150096  0.148143  0.000000  0.148737
            Yes     0.117750  0.124515  0.193370  0.000000  0.139064
       5    No      0.000000  0.000000  0.206928  0.000000  0.206928
            Yes     0.000000  0.106572  0.065660  0.000000  0.086116
       6    No      0.000000  0.000000  0.103799  0.000000  0.103799
Lunch  1    No      0.000000  0.000000  0.000000  0.181728  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000  0.223776
       2    No      0.000000  0.000000  0.000000  0.166005  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843  0.165266
       3    No      0.187735  0.000000  0.000000  0.084246  0.118742
            Yes     0.000000  0.000000  0.000000  0.204952  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706  0.173706
All                 0.169913  0.153152  0.166897  0.161276  0.160803

## Cross-Tabulations: Crosstab

In [112]:
pd.crosstab(tips.smoker, tips.time, margins=True)

time,Dinner,Lunch,All
smoker,,,
No,106,45,151
Yes,70,23,93
All,176,68,244


In [113]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244